# Solving Cartpole with Q-Learning (Bellman)

## Env Setup

In [23]:
import gymnasium as gym
import numpy as np

In [24]:
env = gym.make('CartPole-v1')

## Step and Render for random sample actions

In [ ]:
episodeNumber = 10
timeSteps = 5000
action = -1

for episodeIndex in range(episodeNumber):
    initial_state = env.reset()
    print("Episode: ", episodeIndex)
    env.render()
    for timeIndex in range(timeSteps):
        print("TimeStep:", timeIndex)
        if action == 1:
            random_action = 0
        else:
            random_action = 1
        action *= -1
        observation, reward, terminated, truncated, info = env.step(random_action)
        if terminated:
            break

env.close()


Episode:  0
TimeStep: 0
TimeStep: 1
TimeStep: 2
TimeStep: 3
TimeStep: 4
TimeStep: 5
TimeStep: 6
TimeStep: 7
TimeStep: 8
TimeStep: 9
TimeStep: 10
TimeStep: 11
TimeStep: 12
TimeStep: 13
TimeStep: 14
TimeStep: 15
TimeStep: 16
TimeStep: 17
TimeStep: 18
TimeStep: 19
TimeStep: 20
TimeStep: 21
TimeStep: 22
TimeStep: 23
TimeStep: 24
TimeStep: 25
TimeStep: 26
TimeStep: 27
TimeStep: 28
TimeStep: 29
TimeStep: 30
TimeStep: 31
TimeStep: 32
TimeStep: 33
TimeStep: 34
TimeStep: 35
TimeStep: 36
TimeStep: 37
TimeStep: 38
TimeStep: 39
TimeStep: 40
TimeStep: 41
TimeStep: 42
TimeStep: 43
TimeStep: 44
TimeStep: 45
TimeStep: 46
TimeStep: 47
TimeStep: 48
TimeStep: 49
TimeStep: 50
TimeStep: 51
TimeStep: 52
TimeStep: 53
TimeStep: 54
TimeStep: 55
TimeStep: 56
TimeStep: 57
TimeStep: 58
TimeStep: 59
TimeStep: 60
TimeStep: 61
TimeStep: 62
TimeStep: 63
TimeStep: 64
TimeStep: 65
TimeStep: 66
TimeStep: 67
TimeStep: 68
TimeStep: 69
TimeStep: 70
TimeStep: 71
TimeStep: 72
TimeStep: 73
TimeStep: 74
TimeStep: 75
TimeStep: 

/Users/srujanrao/Library/Python/3.9/lib/python/site-packages/gymnasium/envs/classic_control/cartpole.py:215: UserWarning: WARN: You are calling render method without specifying any render mode. You can specify the render_mode at initialization, e.g. gym.make("CartPole-v1", render_mode="rgb_array")
  gym.logger.warn(


## Q-Learning Algorithm

Bins are used to discretize the state space of the environment. Otherwise, the state space would be too large to effectively store a Q-matrix for all possible states in memory. Q-learning would not be possible due to the virtually infinite number of values the state space could take.

- **alpha**: step size
- **gamma**: discount factor
- **epsilon**: parameter for epsilon-greedy policy
- **number of bins**: 4D value that looks like [position_bin, velocity_bin, angle_bin, angular_velocity_bin]

In [25]:
class QLearning:
    def __init__(self, env, alpha, gamma, epsilon, numberOfEpisodes, numberOfBins, lowerBounds, upperBounds):
        self.env = env
        self.alpha = alpha
        self.gamma = gamma
        self.epsilon = epsilon 
        self.numberOfActions = env.action_space.n 
        self.numberOfEpisodes = numberOfEpisodes
        self.numberOfBins = numberOfBins
        self.lowerBounds = lowerBounds
        self.upperBounds = upperBounds

        # rewards sum per episode
        self.sumEpisodeRewards=[]

        self.Qmatrix=np.random.uniform(low=0, high=1, size=(numberOfBins[0],numberOfBins[1],numberOfBins[2],numberOfBins[3],self.numberOfActions))

    # convert continuous values of the state into discrete values for Q-learning
    def getDiscreteState(self, state):
        position = state[0]
        velocity = state[1]
        angle = state[2]
        angularVelocity = state[3]

        # creating discrete bins for the continuous state space
        cartPositionBin = np.linspace(self.lowerBounds[0],self.upperBounds[0],self.numberOfBins[0])
        cartVelocityBin = np.linspace(self.lowerBounds[1],self.upperBounds[1],self.numberOfBins[1])
        poleAngleBin = np.linspace(self.lowerBounds[2],self.upperBounds[2],self.numberOfBins[2])
        poleAngularVelocityBin = np.linspace(self.lowerBounds[3],self.upperBounds[3],self.numberOfBins[3])

        # get indexs of the bins to which the continuous variables belong to
        indexPosition = np.maximum(np.digitize(position, cartPositionBin)-1,0)
        indexVelocity = np.maximum(np.digitize(velocity, cartVelocityBin)-1,0)
        indexAngle = np.maximum(np.digitize(angle, poleAngleBin)-1,0)
        indexAngularVelocity = np.maximum(np.digitize(angularVelocity, poleAngularVelocityBin)-1,0)

        return (indexPosition, indexVelocity, indexAngle, indexAngularVelocity)

    
    def selectAction(self, state, episodeNumber):

        # enabling random actions for exploration
        if episodeNumber < 500:
            return np.random.choice(self.numberOfActions)
        # eventually decreasing the value for epsilon to make the algorithm more greedy
        elif episodeNumber > 7000:
            self.epsilon = 0.999 * self.epsilon
        
        randomNumber = np.random.random()
        discreteState = self.getDiscreteState(state)

        if randomNumber < self.epsilon:
            return np.random.choice(self.numberOfActions)
        else:
            # select an action such that the Q-value for that action, state pair is the highest possible value in that state
            # np.max(self.Qmatrix[discreteState]))[0] -- will return a list of actions, as there could be multiple possible max actions
            return np.random.choice(np.where(self.Qmatrix[discreteState] == np.max(self.Qmatrix[discreteState]))[0])


    # simulating episodes with Q-learning
    def runEpisodes(self):
        for episodeIndex in range(self.numberOfEpisodes):

            state_S, _ = self.env.reset()
            state_S = list(state_S)

            episodeRewards = []

            print("Episode: ", episodeIndex)

            terminal_state = False
            for _ in range(200):
                state_S_discrete = self.getDiscreteState(state_S)
                action_a = self.selectAction(state_S, episodeIndex)

                state_S_prime, reward, terminal_state, _, _ = self.env.step(action_a)
                episodeRewards.append(reward)

                state_S_prime = list(state_S_prime)
                state_S_prime_discrete = self.getDiscreteState(state_S_prime)

                Q_max = np.max(self.Qmatrix[state_S_prime_discrete])

                if not terminal_state:
                    # update Q-values for non-terminal states
                    diff = reward + self.gamma*Q_max - self.Qmatrix[state_S_discrete + (action_a, )]
                    self.Qmatrix[state_S_discrete + (action_a, )] += self.alpha*diff
                else:
                    diff = reward - self.Qmatrix[state_S_discrete + (action_a, )]
                    self.Qmatrix[state_S_discrete + (action_a, )] += self.alpha*(reward - self.Qmatrix[state_S_discrete + (action_a, )])
                
                state_S = state_S_prime

                if terminal_state:
                    break
            
            
            print("Rewards: ", np.sum(episodeRewards))
            self.sumEpisodeRewards.append(np.sum(episodeRewards))

        
    # final optimal policy using the Q-matrix generated by running Q-learning
    def runOptimalPolicy(self):
        print("Running optimal policy")
        env1 = gym.make('CartPole-v1', render_mode='human')
        curr_state, _ = env1.reset()
        env1.render()

        timeSteps = 1000

        for timeIndex in range(timeSteps):
            print("TimeStep:", timeIndex)
            curr_state_discrete = self.getDiscreteState(curr_state)
            # np.max(self.Qmatrix[discreteState]))[0] -- will return a list of actions, as there could be multiple possible max actions
            optimal_action = np.random.choice(np.where(self.Qmatrix[curr_state_discrete] == np.max(self.Qmatrix[curr_state_discrete]))[0])

            curr_state, reward, terminated, _, _ = env1.step(optimal_action)

            if (terminated):
                print(terminated)
                break
    

## Running Q-Learning on Env

In [26]:
env=gym.make('CartPole-v1')
state, _ = env.reset()

print(state)

[ 0.03817403 -0.03741619 -0.04755735  0.03427091]


### Defining bounds and bins

In [27]:
upperBounds=env.observation_space.high
lowerBounds=env.observation_space.low
cartVelocityMin=-3
cartVelocityMax=3
poleAngleVelocityMin=-10
poleAngleVelocityMax=10
upperBounds[1]=cartVelocityMax
upperBounds[3]=poleAngleVelocityMax
lowerBounds[1]=cartVelocityMin
lowerBounds[3]=poleAngleVelocityMin

print(upperBounds)
print(lowerBounds)

[4.8000002e+00 3.4028235e+38 4.1887903e-01 3.4028235e+38]
[-4.8000002e+00 -3.4028235e+38 -4.1887903e-01 -3.4028235e+38]


In [28]:
num_bins_position = 30
num_bins_velocity = 30
num_bins_angle = 30
num_bins_angular_velocity = 30

num_bins = [num_bins_position, num_bins_velocity, num_bins_angle, num_bins_angular_velocity]

### Defining parameters

In [29]:
alpha = 0.1
gamma = 0.9
epsilon = 0.2
numberOfEpisodes = 20000

In [30]:
Q = QLearning(env, alpha, gamma, epsilon, numberOfEpisodes, num_bins, lowerBounds, upperBounds)

### Running Q-learning on the state specified above

In [31]:
Q.runEpisodes()

Episode:  0
Rewards:  11.0
Episode:  1
Rewards:  32.0
Episode:  2
Rewards:  31.0
Episode:  3
Rewards:  22.0
Episode:  4
Rewards:  14.0
Episode:  5
Rewards:  31.0
Episode:  6
Rewards:  15.0
Episode:  7
Rewards:  16.0
Episode:  8
Rewards:  13.0
Episode:  9
Rewards:  13.0
Episode:  10
Rewards:  34.0
Episode:  11
Rewards:  12.0
Episode:  12
Rewards:  12.0
Episode:  13
Rewards:  22.0
Episode:  14
Rewards:  36.0
Episode:  15
Rewards:  14.0
Episode:  16
Rewards:  53.0
Episode:  17
Rewards:  33.0
Episode:  18
Rewards:  21.0
Episode:  19
Rewards:  38.0
Episode:  20
Rewards:  34.0
Episode:  21
Rewards:  9.0
Episode:  22
Rewards:  24.0
Episode:  23
Rewards:  18.0
Episode:  24
Rewards:  10.0
Episode:  25
Rewards:  15.0
Episode:  26
Rewards:  19.0
Episode:  27
Rewards:  24.0
Episode:  28
Rewards:  16.0
Episode:  29
Rewards:  12.0
Episode:  30
Rewards:  27.0
Episode:  31
Rewards:  27.0
Episode:  32
Rewards:  29.0
Episode:  33
Rewards:  14.0
Episode:  34
Rewards:  12.0
Episode:  35
Rewards:  40.0
Epi

### Running optimal policy on the state specified above

In [32]:
Q.runOptimalPolicy()

Running optimal policy
TimeStep: 0
TimeStep: 1
TimeStep: 2
TimeStep: 3
TimeStep: 4
TimeStep: 5
TimeStep: 6
TimeStep: 7
TimeStep: 8
TimeStep: 9
TimeStep: 10
TimeStep: 11
TimeStep: 12
TimeStep: 13
TimeStep: 14
TimeStep: 15
TimeStep: 16
TimeStep: 17
TimeStep: 18
TimeStep: 19
TimeStep: 20
TimeStep: 21
TimeStep: 22
TimeStep: 23
TimeStep: 24
TimeStep: 25
TimeStep: 26
TimeStep: 27
TimeStep: 28
TimeStep: 29
TimeStep: 30
TimeStep: 31
TimeStep: 32
TimeStep: 33
TimeStep: 34
TimeStep: 35
TimeStep: 36
TimeStep: 37


/Users/srujanrao/Library/Python/3.9/lib/python/site-packages/gymnasium/envs/classic_control/cartpole.py:180: UserWarning: WARN: You are calling 'step()' even though this environment has already returned terminated = True. You should always call 'reset()' once you receive 'terminated = True' -- any further steps are undefined behavior.
  logger.warn(


TimeStep: 38
TimeStep: 39
TimeStep: 40
TimeStep: 41
TimeStep: 42
TimeStep: 43
TimeStep: 44
TimeStep: 45
TimeStep: 46
TimeStep: 47
TimeStep: 48
TimeStep: 49
TimeStep: 50
TimeStep: 51
TimeStep: 52
TimeStep: 53
TimeStep: 54
TimeStep: 55
TimeStep: 56
TimeStep: 57
TimeStep: 58
TimeStep: 59
TimeStep: 60
TimeStep: 61
TimeStep: 62
TimeStep: 63
TimeStep: 64
TimeStep: 65
TimeStep: 66
TimeStep: 67
TimeStep: 68
TimeStep: 69
TimeStep: 70
TimeStep: 71
TimeStep: 72
TimeStep: 73
TimeStep: 74
TimeStep: 75
TimeStep: 76
TimeStep: 77
TimeStep: 78
TimeStep: 79
TimeStep: 80
TimeStep: 81
TimeStep: 82
TimeStep: 83
TimeStep: 84
TimeStep: 85
TimeStep: 86
TimeStep: 87
TimeStep: 88
TimeStep: 89
TimeStep: 90
TimeStep: 91
TimeStep: 92
TimeStep: 93
TimeStep: 94
TimeStep: 95
TimeStep: 96
TimeStep: 97
TimeStep: 98
TimeStep: 99
TimeStep: 100
TimeStep: 101
TimeStep: 102
TimeStep: 103
TimeStep: 104
TimeStep: 105
TimeStep: 106
TimeStep: 107
TimeStep: 108
TimeStep: 109
TimeStep: 110
TimeStep: 111
TimeStep: 112
TimeStep: 11

KeyboardInterrupt: 